In [1]:
try:
    %load_ext autoreload
    %autoreload 2
except:
        pass
%matplotlib inline
import functions.add_path

import functions.plotnine_theme
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from plotnine import *
from plotnine.data import *
import os
print(os.getcwd())
if os.getcwd()[-4:] != 'code':
    try:
        os.chdir('./code')
    except:
        os.chdir('/net/store/nbp/users/behinger/projects/etcomp/code')

import av # import to import before any pupillabs libraries

/net/store/nbp/users/behinger/projects/etcomp/code


In [2]:
import ctypes
ctypes.cdll.LoadLibrary('/net/store/nbp/users/behinger/projects/etcomp/local/build/build_ceres/lib/libceres.so.2')

<CDLL '/net/store/nbp/users/behinger/projects/etcomp/local/build/build_ceres/lib/libceres.so.2', handle 43c9d00 at 0x7f54824fd4e0>

In [3]:
import pupil_producers as pupil_produce
import pupil_detectors

In [4]:
from functions.pl_surface import fake_gpool_surface

reached exception:pyaudio
reached exception:optimization_calibration
reached exception:glfw.GLFW_PRESS
reached exception:glfw.GLFW_KEY_SPACE
reached exception:glfw.GLFW_KEY_F
reached exception:glfw.glfwSetKeyCallback
reached exception:fingertip_calibration
reached exception:glfw.GLFW_PRESS
reached exception:natural_features_calibration
reached exception:optimization_calibration
reached exception:optimization_calibration
reached exception:glfw.GLFW_PRESS
reached exception:glfw.GLFW_KEY_SPACE
reached exception:glfw.GLFW_KEY_F
reached exception:glfw.glfwSetKeyCallback
reached exception:fingertip_calibration
reached exception:glfw.GLFW_PRESS
reached exception:natural_features_calibration
reached exception:optimization_calibration


In [14]:
cap.seek_to_frame(1000)

frame= cap.get_frame()

plt.imshow(frame.gray)

In [41]:

def nbp_pupildetect(eye_id = None,folder=None,options = None,detector_type='2D'):
    #options = {"pupil_size_min":10,'pupil_size_max':150}
    
    import av
    import pl_anna_tools
    from ui_roi import UIRoi
    
    cap = pl_anna_tools.init_playback(video_name = 'eye0.mp4',video_file_path = folder)

    
    #result = detector.detect(frame, UIRoi([frame.height,frame.width]), 0)
    if detector_type == '2D':
        detector = pupil_detectors.detector_2d.Detector_2D()
    elif detector_type == '3D':
        detector = pupil_detectors.detector_3d.Detector_3D()
    else:
        raise('unknown detectortype')
        
    if options:
        # XXX to be implemented after looking at what these parameters do.........
        for key in options.keys():
            assert(key in detector.detectProperties.keys(),'option not found')


    eye_id = 0
    results = [];
    for k in range(100):#range(cap.get_frame_count()):
        cap.seek_to_frame(k)
        frame = cap.get_frame()
        result = detector.detect(frame, UIRoi([frame.height,frame.width]), 0)
        if result is not None:
            result['id'] = eye_id
        results.append(result)
    return(results)

<ipython-input-41-6cc3135bc993>:23: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(key in detector.detectProperties.keys(),'option not found')


In [42]:
pupil_positions = nbp_pupildetect(detector_type = '3D',eye_id = 0,folder='/net/store/nbp/projects/IntoTheWild/Daten/Eyetracking/Wild/VP21/2018_05_24/001')

Loading dummy calibration


In [43]:
pupil_positions

[{'circle_3d': {'center': (0.0, -0.0, 0.0),
   'normal': (0.0, -0.0, 0.0),
   'radius': 0.0},
  'confidence': 0.0,
  'diameter': 0.0,
  'diameter_3d': 0.0,
  'ellipse': {'angle': 90.0, 'axes': (0.0, 0.0), 'center': (320.0, 240.0)},
  'id': 0,
  'method': '3d c++',
  'model_birth_timestamp': -1.0,
  'model_confidence': 0.0,
  'model_id': 1,
  'norm_pos': (0.5, 0.5),
  'phi': 0,
  'projected_sphere': {'angle': 90.0,
   'axes': (0.0, 0.0),
   'center': (0.0, 0.0)},
  'sphere': {'center': (0.0, -0.0, 0.0), 'radius': 0.0},
  'theta': 0,
  'timestamp': 1364.888598,
  'topic': 'pupil'},
 {'circle_3d': {'center': (0.0, -0.0, 0.0),
   'normal': (0.0, -0.0, 0.0),
   'radius': 0.0},
  'confidence': 0.0,
  'diameter': 0.0,
  'diameter_3d': 0.0,
  'ellipse': {'angle': 90.0, 'axes': (0.0, 0.0), 'center': (320.0, 240.0)},
  'id': 0,
  'method': '3d c++',
  'model_birth_timestamp': -1.0,
  'model_confidence': 0.0,
  'model_id': 1,
  'norm_pos': (0.5, 0.5),
  'phi': 0,
  'projected_sphere': {'angle': 9